In [2]:
### import 資料進來
import os, json, pandas as pd

base_dir = "/Users/Andypon/10_交大研究所/1141_01_機器學習與金融科技/data"
file_name = "mcc_codes.json"
file_path = os.path.join(base_dir, file_name)

with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 如果是 dict of scalar → 轉成 [{"code": k, "desc": v}, ...]
if isinstance(data, dict):
    df_mcc = pd.DataFrame([{"code": k, "desc": v} for k, v in data.items()])
else:
    df_mcc = pd.DataFrame(data)

print(df_mcc.head())

   code                                  desc
0  5812         Eating Places and Restaurants
1  5541                      Service Stations
2  7996  Amusement Parks, Carnivals, Circuses
3  5411          Grocery Stores, Supermarkets
4  4784                 Tolls and Bridge Fees


In [ ]:
#cars one heart encoding

SyntaxError: invalid syntax (ipython-input-3595165742.py, line 1)